# **Pre-processing**

### **Import Libraries**

In [34]:
import pandas as pd
import re
import unicodedata
from underthesea import word_tokenize
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
tqdm.pandas()

### **Load data**

In [25]:
df = pd.read_csv('../Data/vietnamese_news_cleaned_raw.csv')

### **Chuẩn hóa Unicode**

In [5]:
def standardize_unicode(text):
    return unicodedata.normalize('NFC', text)

### **Làm sạch văn bản**

In [6]:
def clean_text_patterns(text):
    text = re.sub(r'<[^>]*>', ' ', text)
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'\S*@\S*\s?', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    return text

### **Chuẩn hóa định dạng văn bản**

In [27]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

### **Tách Token**

In [29]:
def tokenizer_pipeline(text):
    try:
        return word_tokenize(text, format='text')
    except Exception as e:
        print(f"Error tokenizing text: {e}")
        return ""
    

### **Loại bỏ stopword**

In [19]:
def load_stopwords(filepath):
    stopwords = set()
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    # Thêm biến thể với dấu gạch dưới
                    word_compatible = word.replace(' ', '_')
                    stopwords.add(word_compatible)
                    stopwords.add(word)
        return stopwords
    except Exception as e:
        print(f"Lỗi trong quá trình đọc file: {e}")
        return set()


In [20]:
stopword_file_path = '../vietnamese-stopwords.txt'
vn_stopwords = load_stopwords(stopword_file_path)

In [21]:
def remove_stopwords(text):
    if not text:
        return ""
    words = text.split()
    clean_words = [w for w in words if w not in vn_stopwords]
    return " ".join(clean_words)

### **Thực hiện pipeline Pre-processing**

In [30]:
def process_vietnamese_text(text):
    if not isinstance(text, str) or text is None:
        return ""
    text = standardize_unicode(text)
    text = clean_text_patterns(text)
    text = normalize_text(text)
    text = tokenizer_pipeline(text)
    text = remove_stopwords(text)
    return text

In [31]:
df['clean_content'] = df['content'].progress_apply(process_vietnamese_text)
df['clean_title'] = df['title'].progress_apply(process_vietnamese_text)

100%|██████████| 30110/30110 [00:55<00:00, 543.23it/s]


In [32]:
sample = df[['content', 'clean_content']].sample(5)

for index, row in sample.iterrows():
    print("-" * 50)
    print(f"GỐC: {row['content'][:100]}...")
    print(f"XỬ LÝ: {row['clean_content'][:100]}...")

--------------------------------------------------
GỐC: Trong cuộc họp báo ngày 9/11, người phát ngôn Bộ Ngoại giao Phạm Thu Hằng thông báo "đến nay đã có 1...
XỬ LÝ: họp_báo người_phát_ngôn ngoại_giao phạm_thu hằng thông_báo công_dân việt_nam trở an_toàn chuyến bay ...
--------------------------------------------------
GỐC: Tình huống giao thông: Đang ở tốc độ gần 80 km/h tại làn đường sát dải phân cách, lái xe có gắn came...
XỬ LÝ: tình_huống giao_thông tốc_độ km h làn_đường sát dải_phân_cách lái_xe gắn camera hành_trình phanh chá...
--------------------------------------------------
GỐC: Khoảng 9h ngày 9/9, tàu container Mississippi neo đậu ở bến G, cảng Long Beach, bị nghiêng sang một ...
XỬ LÝ: h tàu container mississippi neo_đậu bến g cảng long beach nghiêng hàng_loạt container rơi xuống_nước...
--------------------------------------------------
GỐC: Từ đầu năm, doanh số của VinFast luôn ở mức trung bình khoảng 11.000 xe một tháng, tạo khoảng cách l...
XỬ LÝ: đầu doanh_số vinfast

In [33]:
df.head(5)

,date,source,title,content,category,url,date_clean,raw_word_count,clean_content,clean_title
0,16/8/2024,VnExpress,Những lỗi cần tránh khi thiết kế giếng trời,Giếng trời là không gian thông suốt từ tầng tr...,Bất động sản,https://vnexpress.net/nhung-loi-can-tranh-khi-...,2024-08-16,731,giếng trời không_gian thông_suốt tầng mái phươ...,lỗi thiết_kế giếng trời
1,19/12/2023,VnExpress,Căn hộ thông tầng 240 m2 hướng sông Hồng với n...,Căn hộ thông tầng (duplex) có quy mô 2 tầng vớ...,Bất động sản,https://vnexpress.net/can-ho-thong-tang-240-m2...,2023-12-19,858,căn_hộ thông tầng duplex quy_mô tầng tổng diện...,căn_hộ thông tầng m hướng sông hồng nội_thất t...
2,7/11/2024,VnExpress,'Không được kiểm tra hiện trạng công trình khi...,"Nội dung được ông Nguyễn Toàn Thắng, Giám đốc ...",Bất động sản,https://vnexpress.net/khong-duoc-kiem-tra-hien...,2024-11-07,481,nội_dung nguyễn_toàn_thắng giám_đốc sở tài_ngu...,kiểm_tra hiện_trạng công_trình đăng_ký biến_độ...
3,11/6/2024,VnExpress,Hà Nội đề xuất xây mới 9 khu nhà ở xã hội,Thông tin này được Sở Xây dựng Hà Nội nêu tron...,Bất động sản,https://vnexpress.net/ha-noi-de-xuat-xay-moi-9...,2024-06-11,538,thông_tin sở xây_dựng hà_nội nêu báo_cáo tình_...,hà_nội đề_xuất xây khu xã_hội
4,29/1/2025,VnExpress,Biệt thự 700 m2 đưa thiên nhiên vào trung tâm,"Căn biệt thự có quy mô hai tầng, được xây dựng...",Bất động sản,https://vnexpress.net/biet-thu-700-m2-dua-thie...,2025-01-29,732,biệt_thự quy_mô hai tầng xây_dựng khu đất rộng...,biệt thự m thiên_nhiên trung_tâm


### **Lưu data đã pre-processing**

In [36]:
df.to_csv('../Data/vietnamese_news_preprocessed.csv', index=False)